In [2]:
import PyPDF2
from pdf2image import convert_from_path
from PIL import Image
import pytesseract
import pandas as pd
from pytesseract import image_to_string
import os
import re

### Download
* Poppler: https://github.com/oschwartz10612/poppler-windows/releases
* Tesseract: https://github.com/UB-Mannheim/tesseract/wiki

In [62]:
pytesseract.pytesseract.tesseract_cmd = 'F:/Tesseract-OCR/tesseract'
poppler_path="F:/poppler-23.11.0/Library/bin/"

In [3]:
## Fetching PDF files

pdfs=[]
for file in os.listdir():
    if '.pdf' in file:
        pdfs.append(file)

In [23]:
# Getting certificate license code from PDF

def get_code(pdf):
    with open(pdf, 'rb') as file:
        fileReader = PyPDF2.PdfReader(file)
        if fileReader.is_encrypted:
            fileReader.decrypt('')
        # page=fileReader.getPage(0)
        page=fileReader.pages[0]
        page_content = page.extract_text().split()

    code=[i for i in page_content if '/verify' in i][0].split('/verify')[1]
    if 'Coursera' in code:
        code=code[:code.index('Coursera')]
    return code

In [57]:
def pdf_to_image(pdf):
    pages = convert_from_path(pdf, 500, poppler_path=poppler_path)
    for page in pages:
        page.save('out.png', 'PNG')
    image=Image.open('out.png')
    return image

In [7]:
# Fetching Date, person, Course Name, Organization, URL from the PDF

def course_details(image,code):
    text=image_to_string(image)
    date_index=-1
    for line in text.split('\n'):
        date_index+=1
        if '/' in line:
            break

    text_list=text.split('\n')[date_index:]
    text_list=[i for i in text_list if ('' != i)]
    text_list=[i for i in text_list if (' ' != i)]

    new_text=' '.join(text_list)

    date=text_list[0]
    print(date)

    link='https://www.'+(new_text.split('/verify')[0]).split()[-1]+'/account/accomplishments/certificate'+(new_text.split('/verify')[1]).split()[0]
    if code!=(new_text.split('/verify')[1]).split()[0]:
        other_link='https://www.'+(new_text.split('/verify')[0]).split()[-1]+'/account/accomplishments/certificate/'+code
    else:
        other_link=None
    print(link,other_link)

    course_pattern1=r'completed(.+)a \d week'
    course_pattern2=r'completed(.+)an online'
    course_pattern3=r'distinction(.+)a 4 week'
    course_pattern4=r'distinction(.+)an online'

    course_regex1=re.compile(course_pattern1)
    course_regex2=re.compile(course_pattern2)
    course_regex3=re.compile(course_pattern3)
    course_regex4=re.compile(course_pattern4)

    course_search1=course_regex1.search(new_text)
    course_search2=course_regex2.search(new_text)
    course_search3=course_regex3.search(new_text)
    course_search4=course_regex4.search(new_text)

    if course_search3:
        course=course_search3.groups()[0].strip()
    elif course_search4:
        course=course_search4.groups()[0].strip()
    elif course_search1:
        course=course_search1.groups()[0].strip()
    elif course_search2:
        course=course_search2.groups()[0].strip()
    else:
        course=None
    print(course)

    organization_pattern=r'authorized by(.+)and offered'
    organization_regex=re.compile(organization_pattern)
    organization_search=organization_regex.search(new_text)
    organization=organization_search.groups()[0].strip()
    print(organization)

    name=' '.join([i for i in text_list[1].split() if i.isupper()][:2])
    print(name)
    print()

    return [date,name,course,organization,link,other_link]

In [65]:
# Extracting information from all PDFs

info=[]
error_pdfs=[]
for pdf in pdfs:
    try:
        code=get_code(pdf)
        image=pdf_to_image(pdf)
        info.append(course_details(image,code)+[pdf])
    except:
        print(pdf)
        print()
        error_pdfs.append(pdf)\
        
        continue

08/17/2018
https://www.coursera.org/account/accomplishments/certificate/2MW8VE9SPVHL None
Agile with Atlassian Jira
Atlassian
SHAURYA KHURANA

10/11/2019
https://www.coursera.org/account/accomplishments/certificate/KLT83QMUTE3DQ https://www.coursera.org/account/accomplishments/certificate//KLT8QMUTE3DQ
AI Workflow: Data Analysis and Hypothesis Testing
IBM
SHAURYA KHURANA

10/15/2019
https://www.coursera.org/account/accomplishments/certificate/6WD5S5Q0JD3F8 https://www.coursera.org/account/accomplishments/certificate//6WD5S5QJD3F8
AI Workflow: Feature Engineering and Bias Detection
IBM
SHAURYA KHURANA

11/20/2018
https://www.coursera.org/account/accomplishments/certificate/5SEAGTSLUCGHD https://www.coursera.org/account/accomplishments/certificate//5EAGTSLUCGHD
AWS Fundamentals: Going Cloud-Native
Amazon Web Services
SHAURYA KHURANA

Date of Issue: 01/11/2018
https://www.credly.com/account/accomplishments/certificate/dc7661 None
None
beta test.pdf

06/14/2015
https://www.coursera.org/acc

Verify at coursera.org/verify/BPHPQCSPZWG6
https://www.coursera.org/account/accomplishments/certificate/BPHPQCSPZWG6 None
None
Image Processing with Python.pdf

06/13/2015
https://www.coursera.org/account/accomplishments/certificate/VM82DDS2WTL3 None
Initiating and Planning Projects
University of California, Irvine
SHAURYA KHURANA

Verify at coursera.org/verify/BJQJVNNX5YF3
https://www.coursera.org/account/accomplishments/certificate/BJQJVNNX5YF3 None
None
Introduction to Git and GitHub.pdf

03/27/2018
https://www.coursera.org/account/accomplishments/certificate/HZT2DFZCPTYS None
Introduction to Graph Theory
University of California, San Diego and National Research University Higher School of Economics
SHAURYA KHURANA

01/27/2016
https://www.coursera.org/account/accomplishments/certificate/MNMERT3W72XA None
Introduction to HTMLs5
University of Michigan
SHAURYA KHURANA

12/16/2015
https://www.coursera.org/account/accomplishments/certificate/RON3M96WYE https://www.coursera.org/account/ac

In [66]:
error_pdfs

['beta test.pdf',
 'Build a Data Science Web App with Streamlit.pdf',
 'Build a film club web app on Google AppEngine.pdf',
 'Build a Machine Learning Web App with Streamlit.pdf',
 'Computer Vision - Image Basics with OpenCV and Python.pdf',
 'Computer Vision - Object Detection with OpenCV and Python.pdf',
 'Computer Vision - Object Tracking.pdf',
 'Create a Python Application using MySQL.pdf',
 'Create Docker Container with Flask Seaborn Regression Plot App.pdf',
 'Create Python Linux Script to Generate a Disk Usage Report.pdf',
 'Create Your First Chatbot with Rasa and Python.pdf',
 'Create Your First Web App with Python and Flask.pdf',
 'Data Visualization with Plotly Express.pdf',
 'Dataplex by Google Cloud.pdf',
 'Detecting COVID-19 with Chest X-Ray using PyTorch.pdf',
 'Earsketch.pdf',
 'GitHub CoPilot.pdf',
 'Google IT Support Specialization.pdf',
 'Image Processing with Python.pdf',
 'Introduction to Git and GitHub.pdf',
 'Python Dynamic HTML Web Server.pdf',
 'SQL for Data Sci

In [73]:
len(info)

55

In [72]:
data=pd.DataFrame(info)
data.columns=['Date','Name','Course','Organization','Link','Alternate Link','Filename']
# data['Date']=pd.to_datetime(data['Date'],format='%m/%d/%Y')
data

,Date,Name,Course,Organization,Link,Alternate Link,Filename
0,08/17/2018,SHAURYA KHURANA,Agile with Atlassian Jira,Atlassian,https://www.coursera.org/account/accomplishmen...,None,Agile with Atlassian Jira.pdf
1,10/11/2019,SHAURYA KHURANA,AI Workflow: Data Analysis and Hypothesis Testing,IBM,https://www.coursera.org/account/accomplishmen...,https://www.coursera.org/account/accomplishmen...,AI Workflow- Data Analysis and Hypothesis.pdf
2,10/15/2019,SHAURYA KHURANA,AI Workflow: Feature Engineering and Bias Dete...,IBM,https://www.coursera.org/account/accomplishmen...,https://www.coursera.org/account/accomplishmen...,AI Workflow- Feature Engineering and Bias Dete...
3,11/20/2018,SHAURYA KHURANA,AWS Fundamentals: Going Cloud-Native,Amazon Web Services,https://www.coursera.org/account/accomplishmen...,https://www.coursera.org/account/accomplishmen...,AWS Fundamentals- Going Cloud Native.pdf
4,06/14/2015,SHAURYA KHURANA,Budgeting and Scheduling Projects,"University of California, Irvine",https://www.coursera.org/account/accomplishmen...,https://www.coursera.org/account/accomplishmen...,Budgeting and Scheduling Projects.pdf
5,10/21/2016,COURSE,Business Metrics for Data-Driven Companies,Duke University,https://www.coursera.org/account/accomplishmen...,None,Business Metrics for Data-Driven Companies.pdf
6,09/18/2015,SHAURYA KHURANA,Corporate Financial Decision-Making for Value ...,The University of Melbourne,https://www.coursera.org/account/accomplishmen...,None,Corporate Financial Decision-Making for Value ...
7,10/06/2019,SHAURYA KHURANA,Project: Multiple Linear Regression with sciki...,Rhyme,https://www.coursera.org/account/accomplishmen...,https://www.coursera.org/account/accomplishmen...,Coursera Multiple Regression Machine Learning.pdf
8,07/27/2020,SHAURYA KHURANA,Crash Course on Python,Google,https://www.coursera.org/account/accomplishmen...,None,Crash Course on Python.pdf
9,08/21/2020,SHAURYA KHURANA,Create Interactive Dashboards with Streamlit a...,Coursera Project Network,https://www.coursera.org/account/accomplishmen...,None,Create Interactive Dashboards with Streamlit.pdf


In [ ]:
# data.to_csv('Certificates.csv',index=False)